In [1]:
import os
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt

# Input and output folder paths
input_folder_path = r"File_Path_Input"
output_folder_path = r"File_Path_Output"

# Ensure the output folder exists, create it if not
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# Loop through each file in the input folder
for file_name in os.listdir(input_folder_path):
    if file_name.endswith(".csv"):
        # Build the full file paths
        file_path = os.path.join(input_folder_path, file_name)
        output_file_path = os.path.join(output_folder_path, f"{os.path.splitext(file_name)[0]}_FORECAST.csv")

        # Load the CSV file into a DataFrame with a specified encoding
        data = pd.read_csv(file_path, encoding='ISO-8859-1')

        # Convert 'Date' to datetime type
        data['Date'] = pd.to_datetime(data['Date'])

        # List of columns to forecast
        forecast_columns = [
            'Minimum Temperature (°C)',
            'Maximum Temperature (°C)',
            'Minimum Relative Humidity (%)',
            'Maximum Relative Humidity (%)',
            'Rainfall (mm)',
            'Average 10m Wind Speed (m/sec)',
            'Evapo-Transpiration (mm)',
            'Solar Radiation (MJ/sq m)'
        ]

        # Convert numeric columns to numeric data type
        numeric_columns = [
            'Minimum Temperature (°C)',
            'Maximum Temperature (°C)',
            'Minimum Relative Humidity (%)',
            'Maximum Relative Humidity (%)',
            'Rainfall (mm)',
            'Average 10m Wind Speed (m/sec)',
            'Evapo-Transpiration (mm)',
            'Solar Radiation (MJ/sq m)'
        ]

        # Remove unwanted columns (Latitude, Longitude, Station Point, State)
        data = data[['Date', 'Station Name'] + forecast_columns]

        data[numeric_columns] = data[numeric_columns].apply(pd.to_numeric, errors='coerce')

        # Replace null values with the value from the row above
        data[numeric_columns] = data[numeric_columns].fillna(method='ffill')

        # Initialize an empty list to store DataFrames and Figures
        forecast_dfs = []

        # Loop through each column and create a separate forecast
        for column in forecast_columns:
            # Prepare the data in the required format for Prophet
            prophet_data = data[['Date', 'Station Name', column]].rename(columns={'Date': 'ds', 'Station Name': 'station', column: 'y'})

            # Initialize the Prophet model
            model = Prophet()

            # Fit the model with the data
            model.fit(prophet_data)

            # Create a DataFrame with future dates for prediction
            future = model.make_future_dataframe(periods=365 * 5)  # 5 years forecast

            # Generate the forecast
            forecast = model.predict(future)

            # Append the forecast DataFrame to the list
            forecast_dfs.append(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].rename(columns={'yhat': f'{column} Forecast', 'yhat_lower': f'{column} Lower', 'yhat_upper': f'{column} Upper'}))

        # Combine all forecasts into a single DataFrame
        forecast_combined = pd.concat([df.set_index('ds') for df in forecast_dfs], axis=1)

        # Reset the index to make 'ds' a regular column
        forecast_combined = forecast_combined.reset_index()

        # Merge the 'station' and the forecasted DataFrames on the index
        forecast_combined = pd.merge(data[['Date', 'Station Name']], forecast_combined, left_on='Date', right_on='ds', how='right')

        # Drop the redundant 'ds' column
        forecast_combined = forecast_combined.drop(columns=['ds'])

        # Save the combined forecast DataFrame to a CSV file in the Forecast folder
        forecast_combined.to_csv(output_file_path, index=False)

        print(f"Forecast saved for {file_name}")

print("Forecasting completed.")


C:\Users\FelixRalphs\anaconda3\envs\WeatherForecast\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
C:\Users\FelixRalphs\AppData\Local\Temp\ipykernel_19084\2334411561.py:57: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data[numeric_columns] = data[numeric_columns].fillna(method='ffill')
16:57:50 - cmdstanpy - INFO - Chain [1] start processing
16:57:51 - cmdstanpy - INFO - Chain [1] done processing
16:57:52 - cmdstanpy - INFO - Chain [1] start processing
16:57:53 - cmdstanpy - INFO - Chain [1] done processing
16:57:55 - cmdstanpy - INFO - Chain [1] start processing
16:57:55 - cmdstanpy - INFO - Chain [1] done processing
16:57:57 - cmdstanpy - INFO - Chain [1] start p

Forecast saved for ARCHERFIELD AIRPORT-BOM-data.csv


C:\Users\FelixRalphs\AppData\Local\Temp\ipykernel_19084\2334411561.py:57: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data[numeric_columns] = data[numeric_columns].fillna(method='ffill')
16:58:06 - cmdstanpy - INFO - Chain [1] start processing
16:58:07 - cmdstanpy - INFO - Chain [1] done processing
16:58:08 - cmdstanpy - INFO - Chain [1] start processing
16:58:09 - cmdstanpy - INFO - Chain [1] done processing
16:58:10 - cmdstanpy - INFO - Chain [1] start processing
16:58:11 - cmdstanpy - INFO - Chain [1] done processing
16:58:12 - cmdstanpy - INFO - Chain [1] start processing
16:58:13 - cmdstanpy - INFO - Chain [1] done processing
16:58:14 - cmdstanpy - INFO - Chain [1] start processing
16:58:14 - cmdstanpy - INFO - Chain [1] done processing
16:58:16 - cmdstanpy - INFO - Chain [1] start processing
16:58:16 - cmdstanpy - INFO - Chain [1] done processing
16:58:17 - cmdstanpy - INFO - Chain [1] s

Forecast saved for BRISBANE AERO-BOM-data.csv


C:\Users\FelixRalphs\AppData\Local\Temp\ipykernel_19084\2334411561.py:57: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data[numeric_columns] = data[numeric_columns].fillna(method='ffill')
16:58:21 - cmdstanpy - INFO - Chain [1] start processing
16:58:22 - cmdstanpy - INFO - Chain [1] done processing
16:58:23 - cmdstanpy - INFO - Chain [1] start processing
16:58:24 - cmdstanpy - INFO - Chain [1] done processing
16:58:25 - cmdstanpy - INFO - Chain [1] start processing
16:58:25 - cmdstanpy - INFO - Chain [1] done processing
16:58:27 - cmdstanpy - INFO - Chain [1] start processing
16:58:27 - cmdstanpy - INFO - Chain [1] done processing
16:58:29 - cmdstanpy - INFO - Chain [1] start processing
16:58:29 - cmdstanpy - INFO - Chain [1] done processing
16:58:31 - cmdstanpy - INFO - Chain [1] start processing
16:58:32 - cmdstanpy - INFO - Chain [1] done processing
16:58:33 - cmdstanpy - INFO - Chain [1] s

...

Forecast saved for SYDNEY AIRPORT AMO-BOM-data.csv


C:\Users\FelixRalphs\AppData\Local\Temp\ipykernel_19084\2334411561.py:57: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data[numeric_columns] = data[numeric_columns].fillna(method='ffill')
17:00:11 - cmdstanpy - INFO - Chain [1] start processing
17:00:12 - cmdstanpy - INFO - Chain [1] done processing
17:00:13 - cmdstanpy - INFO - Chain [1] start processing
17:00:14 - cmdstanpy - INFO - Chain [1] done processing
17:00:16 - cmdstanpy - INFO - Chain [1] start processing
17:00:16 - cmdstanpy - INFO - Chain [1] done processing
17:00:18 - cmdstanpy - INFO - Chain [1] start processing
17:00:19 - cmdstanpy - INFO - Chain [1] done processing
17:00:20 - cmdstanpy - INFO - Chain [1] start processing
17:00:21 - cmdstanpy - INFO - Chain [1] done processing
17:00:22 - cmdstanpy - INFO - Chain [1] start processing
17:00:23 - cmdstanpy - INFO - Chain [1] done processing
17:00:24 - cmdstanpy - INFO - Chain [1] s

Forecast saved for TEWANTIN RSL PARK-BOM-data.csv


C:\Users\FelixRalphs\AppData\Local\Temp\ipykernel_19084\2334411561.py:57: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data[numeric_columns] = data[numeric_columns].fillna(method='ffill')
17:00:28 - cmdstanpy - INFO - Chain [1] start processing
17:00:29 - cmdstanpy - INFO - Chain [1] done processing
17:00:30 - cmdstanpy - INFO - Chain [1] start processing
17:00:31 - cmdstanpy - INFO - Chain [1] done processing
17:00:33 - cmdstanpy - INFO - Chain [1] start processing
17:00:33 - cmdstanpy - INFO - Chain [1] done processing
17:00:35 - cmdstanpy - INFO - Chain [1] start processing
17:00:36 - cmdstanpy - INFO - Chain [1] done processing
17:00:37 - cmdstanpy - INFO - Chain [1] start processing
17:00:37 - cmdstanpy - INFO - Chain [1] done processing
17:00:39 - cmdstanpy - INFO - Chain [1] start processing
17:00:39 - cmdstanpy - INFO - Chain [1] done processing
17:00:41 - cmdstanpy - INFO - Chain [1] s

Forecast saved for TOWNSVILLE AERO-BOM-data.csv
Forecasting completed.
